In [1]:
import pymongo
import os,csv,sys
import openslide

client = pymongo.MongoClient('localhost',27017)
tcga_mirror_root = '/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/'
tcga_slide_db = client['TCGA_Slide_DB']
load_errors_db = client['CDSA_LoadErrors']


In [5]:
svs_collections = [x for x in os.listdir(tcga_mirror_root) if os.path.isdir(os.path.join(tcga_mirror_root,x)) ]
print len(svs_collections),'svs collection sets are available'
print svs_collections

35 svs collection sets are available
['coad', 'uvm', 'sarc', 'acc', 'prad', 'laml', 'paad', 'dlbc', 'lusc', 'thca', 'lgg', 'pcpg', 'luad', 'ucec', 'meso', 'brca', 'blca', 'kirp', 'thym', 'ucs', 'read', 'kirc', 'gbm', 'kich', 'stad', 'chol', 'lcml', 'hnsc', 'ov', 'tgct', 'skcm', 'esca', 'cesc', 'lihc', 'cntl']


In [3]:
import dsa_mongo_common_functions as dsa
import cdsa_loader_helper_functions as cdsa_helpers
reload(dsa)
reload(cdsa_helpers)

<module 'cdsa_loader_helper_functions' from 'cdsa_loader_helper_functions.pyc'>

In [4]:
for collection_id in svs_collections:

    curr_svs_slide_list = dsa.find_svs_image_list(collection_id,[tcga_mirror_root])
    
    slides_processed = 0
    newly_processed = 0
    dup_slide = 0
    rescanned_slides =0 

    for sld in curr_svs_slide_list:
        #print sld
        ## Building the basic dictionary to look up.... for now I'll just index on the full slide name... may change this later though

        slide_name = os.path.basename(sld)
        ### check the DB to see if the slide is indexed already
        qry = tcga_slide_db[collection_id].find_one( {'slide_name':slide_name})
        if not qry:
            #print "Slide is not already indexed....",sld
            fs = os.path.getsize(sld)
            #md5Checksum = dsa.md5sum(sld)
            (openslide_could_open, width, height, filesize, orig_resolution, slide_name,md5, sld_properties) = cdsa_helpers.openslide_test_file_mongo( sld, 'svs', client)
            if openslide_could_open:

                #print width,height,filesize,orig_resolution,md5, sld_properties

                if 'diagnostic_images' in sld:
                    prep_type = 'diagnostic'
                elif 'tissue_images' in sld:
                    prep_type = 'tissue_image'
                else:
                    prep_type = "Unknown"

                slide_metadata = { 'slide_w_path': sld, 'slide_name': slide_name, 'file_size':fs, 'width':width, 'height':height,
                                 'orig_resolution': orig_resolution, 'sld_properties': cdsa_helpers.clean_openslide_keys ( sld_properties), 'slide_md5': md5, 'prep_type': prep_type
                                 }
                #print slide_metadata
                tcga_slide_db[collection_id].insert_one(slide_metadata)
                newly_processed +=1 
            else:
                print "UNABLE TO OPEN FILE??",sld
                #sys.exit()
        else:
            ### See if the other slide matches
            #print "Image seems to have been loaded?"
            fs = os.path.getsize(sld)
            ## Double check if file size matches
            if qry['file_size'] != fs:
                #print "File size mismatch??",fs,qry['file_size'],qry['slide_w_path'],sld
                load_errors_db['rescanned_slides'].insert_one( {'loaded_slide': qry['slide_w_path'], 'rescanned_slide': sld}             )
                rescanned_slides +=1 
            else:
                dup_slide +=1


        slides_processed +=1        
        output = "Total Processed: %d  Newly Processed: %d Dup Slides or Already Loaded: %d  RESCANNED Slides %d" % (slides_processed, newly_processed, dup_slide, rescanned_slides )
        dsa.LinePrinter(output)


/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/
Total Processed: 1267  Newly Processed: 0 Dup Slides or Already Loaded: 1267  RESCANNED Slides 0Openslide returned an error om tje StandardError block /TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/coad/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_COAD.diagnostic_images.Level_1.138.10.0/TCGA-AA-3489-01Z-00-DX1.AE299B70-B14C-4FFE-B1F9-38B2EB267FA9.svs
SHIT IT DIED!
insert into `corrupt_or_unreadable_svs_files` (full_file_name,filesize) Values ('/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/coad/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_COAD.diagnostic_images.Level_1.138.10.0/TCGA-AA-3489-01Z-00-DX1.AE299B70-B14C-4FFE-B1F9-38B2EB267FA9.svs',7610002) 
UNABLE TO OPEN FILE?? /TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonym

Verify failed with: ()
Verify failed with:

Openslide returned an error om tje StandardError block /TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/coad/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_COAD.diagnostic_images.Level_1.138.10.0/TCGA-AA-3511-01Z-00-DX1.F66F89C7-147D-4EE9-A482-61C3033EF443.svs
SHIT IT DIED!
insert into `corrupt_or_unreadable_svs_files` (full_file_name,filesize) Values ('/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/coad/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_COAD.diagnostic_images.Level_1.138.10.0/TCGA-AA-3511-01Z-00-DX1.F66F89C7-147D-4EE9-A482-61C3033EF443.svs',10442472) 
UNABLE TO OPEN FILE?? /TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/coad/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_COAD.diagnostic_images.Level_1.138.10.0/TCGA-AA-3511-01Z-00-DX1.F66F89C7-147D-4EE9-A

 ()
Verify failed with:

Openslide returned an error om tje StandardError block /TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/coad/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_COAD.diagnostic_images.Level_1.138.10.0/TCGA-AA-3496-01Z-00-DX1.B109A6F3-02E0-4181-B69A-00CBA758C074.svs
SHIT IT DIED!
insert into `corrupt_or_unreadable_svs_files` (full_file_name,filesize) Values ('/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/coad/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_COAD.diagnostic_images.Level_1.138.10.0/TCGA-AA-3496-01Z-00-DX1.B109A6F3-02E0-4181-B69A-00CBA758C074.svs',8600540) 
UNABLE TO OPEN FILE?? /TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/coad/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_COAD.diagnostic_images.Level_1.138.10.0/TCGA-AA-3496-01Z-00-DX1.B109A6F3-02E0-4181-B6

...

 ("cannot identify image file '/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/cesc/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_CESC.diagnostic_images.Level_1.127.5.0/TCGA-DS-A1OB-01Z-00-DX1.F3E3B423-D5E3-463A-BE95-76C2ADD14915.svs'",)
Verify failed with:

Openslide returned an error om tje StandardError block /TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/cesc/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_CESC.diagnostic_images.Level_1.127.3.0/TCGA-DS-A1OA-01Z-00-DX1.2452ACCB-9788-44C3-8041-3D9E5A3620BD.svs
SHIT IT DIED!
insert into `corrupt_or_unreadable_svs_files` (full_file_name,filesize) Values ('/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/cesc/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_CESC.diagnostic_images.Level_1.127.3.0/TCGA-DS-A1OA-01Z-00-DX1.2452ACCB-9788-44C3-8041-3D9E5A3620BD.svs',1796904111) 
UNABLE TO OPEN FILE?? /TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/cesc/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_CESC.diagnostic_images.Level_1.127.3.0/TCGA-DS-A1OA-01Z-00-DX1.2452ACCB-9788-44C3-80

 ("cannot identify image file '/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/cesc/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_CESC.diagnostic_images.Level_1.127.3.0/TCGA-DS-A1OA-01Z-00-DX1.2452ACCB-9788-44C3-8041-3D9E5A3620BD.svs'",)
Verify failed with:

Openslide returned an error om tje StandardError block /TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/cesc/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_CESC.diagnostic_images.Level_1.127.3.0/TCGA-DS-A1OB-01Z-00-DX1.F3E3B423-D5E3-463A-BE95-76C2ADD14915.svs
SHIT IT DIED!
insert into `corrupt_or_unreadable_svs_files` (full_file_name,filesize) Values ('/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/cesc/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_CESC.diagnostic_images.Level_1.127.3.0/TCGA-DS-A1OB-01Z-00-DX1.F3E3B423-D5E3-463A-BE95-76C2ADD14915.svs',845977863) 
UNABLE TO OPEN FILE?? /TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/cesc/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_CESC.diagnostic_images.Level_1.127.3.0/TCGA-DS-A1OB-01Z-00-DX1.F3E3B423-D5E3-463A-BE9

 ("cannot identify image file '/TCGA_MIRROR/tcga-data.nci.nih.gov/tcgafiles/ftp_auth/distro_ftpusers/anonymous/tumor/cesc/bcr/nationwidechildrens.org/diagnostic_images/slide_images/nationwidechildrens.org_CESC.diagnostic_images.Level_1.127.3.0/TCGA-DS-A1OB-01Z-00-DX1.F3E3B423-D5E3-463A-BE95-76C2ADD14915.svs'",)


In [ ]:
# collection_id='kirp'
# print svs_collections ## This lists all the top level folders
# curr_svs_slide_list = dsa.find_svs_image_list(collection_id,[tcga_mirror_root])



In [ ]:
### CHECK OUT FILE 2620 +-10 this is for PRAD

In [ ]:
print len(curr_svs_slide_list)

In [ ]:
print curr_svs_slide_list[4]